In [14]:
import hopsworks
from datetime import datetime, timezone

In [15]:
# Connect to the Hopsworks Feature Store
project = hopsworks.login()
fs = project.get_feature_store()

Connection closed.
2024-10-17 23:06:35,064 INFO: Python Engine initialized.

Logged in to project, explore it here https://demo.hops.works/p/123


In [18]:
from hsfs.feature import Feature

# Setup the feature groups for the bytewax pipelines
features = [
    Feature(name="account_id", type="string"),
    Feature(name="date_time", type="timestamp"),
    Feature(name="count_trans", description="Number of transactions in the last 5 minutes", type="bigint"),
    Feature(name="min_amount", description="Min transaction amount in the last 5 minutes", type="double"),
    Feature(name="max_amount", description="Max transaction amount in the last 5 minutes", type="double"),
    Feature(name="avg_amount", description="Average transaction amount in the last 5 minutes", type="double")
]

fg = fs.create_feature_group(
    name="profiles_activity_5m",
    description="Profiles activity within the last 5 minutes",
    version=1,
    primary_key=["account_id"],
    event_time="date_time",
    statistics_config={'histograms': True, 'correlations': True},
    online_enabled=True,
)

fg.save(features)

Feature Group created successfully, explore it at 
https://demo.hops.works/p/123/fs/68/fg/1086


(None, None)

In [19]:
## Set a materialization job for the feature group in Hopsworks
# materialize every 10 minutes
fg.materialization_job.schedule(cron_expression = "0 /10 * ? * * *", start_time=datetime.now(tz=timezone.utc))